Загружаем библиотеки

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Подготавливаем данные для обучения: балансируем количество легальных и мошеннических транзакци, определяем таргет и оставляем поля, которые имеют важную информацию для классификации информацию

In [ ]:
df = pd.read_csv("fraud.csv",sep=',')

features = ['trans_date_trans_time', 'amt', 'category', 'merchant', 'city', 'state', 'job', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'gender']
target = 'is_fraud'

fraud=df[df[target]==1]
not_fraud=df[df[target]==0]
not_fraud_down_sample=not_fraud.sample(n=20000)
under=pd.concat([fraud,not_fraud_down_sample])

x = under[features]
y = under[target]

x['hour']=pd.to_datetime(x['trans_date_trans_time'],dayfirst=True).dt.hour
x=x.drop(['trans_date_trans_time'],axis=1)
categorical_features = ['category', 'merchant', 'city', 'state', 'job', 'gender']
numeric_features = ['amt', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'hour']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
x = preprocessor.fit_transform(x)

Разделяем данные на обучающиеся и тестируемые

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.2)

Определяем гиперпараметры: строгость границы, чувствительность к обучающим примерам, ядро. Затем можно вывести качество предсказаний

In [ ]:
c=SVC()
c_param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10],
    'kernel': ['rbf', 'linear', 'sigmoid']
}
c_grid=GridSearchCV(c,c_param_grid, cv=5 ,n_jobs=-1, verbose=3)
c_grid.fit(x_train,y_train)

In [36]:
best_params=c_grid.best_params_
c_best=SVC(**best_params)
c_best.fit(x_train,y_train)
pred=c_best.predict(x_test)
print(best_params)
print(metrics.classification_report(y_test,pred,digits=5))

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0    0.97096   0.98699   0.97891     15990
           1    0.85773   0.72654   0.78670      1726

    accuracy                        0.96162     17716
   macro avg    0.91435   0.85676   0.88281     17716
weighted avg    0.95993   0.96162   0.96018     17716

